# Лабораторная работа №5
**Вариант №4**\
Решить задачу Дирихле для уравнения Пуассона методом установления с упорядоченным набором оптимальных чебышевских параметров с точнотью $\left| \dfrac{\Delta u}{u} \right|_1 \leqslant 0,001$\
$
\begin{cases}
    \dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2} = 2y(1-y)+ \dfrac{1}{2}(1-4x^2)\\
    u|_{\Gamma} = 0    
\end{cases}
 -\dfrac{1}{2} \leqslant x \leqslant \dfrac{1}{2}, 
0 \leqslant y \leqslant 1
$

In [179]:
import math
import numpy as np
import pandas as pd

### Выбор шага - полиномы Чебышева

In [180]:
def find_sequence(N):
    temp = N
    seq = []; ind = []; flag = 0; count = 0
    while N > 1:
        if N % 2 == 0:
            seq.append(N)
            if (N/2) % 2 == 0:
                flag = 0
            if count == 1 and ind[-1] != 1:
                flag = 1
            ind.append(flag)
            N = int(N/2)
            flag = 0
        if N % 2 != 0:
            seq.append(N); ind.append(flag)
            N = N - 1
            flag = 1
            count += 1
    return seq, ind

In [181]:
seq, ind = find_sequence(10)
print(seq)
print(ind)

[10, 5, 4, 2, 1]
[0, 0, 1, 0, 0]


### Алгоритмы оптимизации

In [182]:
def algorithm_1(arr,n):
    new_arr = []
    for el in arr:
        new_arr.append(el)
        new_arr.append(2*n - el)
        # n -= 1
    return new_arr

In [183]:
def algorithm_2(arr,n):
    m = n//2
    arr.append(4 * m + 1)
    return arr

In [184]:
def algorithm_3(arr,n):
    m = n//2
    new_arr = []
    for el in arr:
        new_arr.append(el)
        new_arr.append(4*m + 2 - el)
    return new_arr

In [185]:
def algorithm_4(arr,n):
    m = n//2
    arr.append(2 * m + 1)
    return arr

In [186]:
def find_step_sequence(N):
    seq, ind = find_sequence(N)
    seq = list(reversed(seq)); ind = list(reversed(ind))
    arr = [1]; prev = 0
    for i in range(1,len(seq)):
        if seq[i] % 2 == 0 and ind[i] != 1:
            arr = algorithm_1(arr,seq[i])
            prev = 1
        if seq[i] % 2 != 0 and ind[i] != 1 and prev != 3 and prev != 4:
            arr = algorithm_2(arr,seq[i])
            prev = 2
        if seq[i] % 2 == 0 and ind[i] == 1:
            arr = algorithm_3(arr,seq[i])
            prev = 3
        if prev == 3:
            arr = algorithm_4(arr,seq[i])
            prev = 4
    return arr

In [187]:
seq = find_step_sequence(10)
print(seq)

[1, 19, 9, 11, 3, 17, 7, 13, 5, 15]


### Вычисление шага

In [188]:
def find_tau(sequence, K, L, N):
    mju_max = 4*(K**2 + L**2)
    mju_min = 2*math.pi**2
    tau = []
    tau_ = lambda n: 2/((mju_max+mju_min)+(mju_max-mju_min)*math.cos(math.pi*(2*n-1)/(2*N)))
    for el in sequence:
        n = (el + 1)/2
        tau.append(tau_(n))
    return tau

### Аналитическое решение

In [189]:
def analytic_solution(x,y):
    return (1 - y) * y* (x**2 - 1/4)

In [190]:
def analytic_points(n):
        x = np.linspace(-1/2,1/2,n)
        y = np.linspace(0,1,n)
        u = np.zeros((n, n))
        for j in range(len(y)):
            for i in range(len(x)):
                u[i][j] = analytic_solution(x[i],y[j])
        return u

### Численное решение

In [191]:
def solve(koef_x,koef_y, N):

    L0 = 6
    K = koef_x * (L0-1) + 1
    L = koef_y * (L0-1) + 1
    seq = find_step_sequence(N)
    tau_list = find_tau(seq, K, L, N)
    prev = np.zeros((K,L))
    next = np.zeros((K,L))
    x, h_x = np.linspace(-1/2, 1/2, K, retstep=True)
    y, h_y = np.linspace(0, 1, L, retstep=True)
    u_ = lambda tau, h_x, h_y, curr, next_x, prev_x, next_y, prev_y, f: curr + tau/(h_x**2) *  (next_x - 2*curr + prev_x) + tau/(h_y**2) *  (next_y - 2*curr + prev_y) - tau*f
    
    # for j in range(L):
    #     for i in range(K):
    #         prev[i][j] = analytic_solution(x[i],y[j])
    
    for t in range(N):
        prev = next.copy()
        tau = tau_list[t]
        for i in range(1,K-1):
            for j in range(1,L-1):
                f = 2*y[j]*(1-y[j])+1/2*(1-4*x[i]**2)
                next[i][j] = u_(tau,h_x,h_y,prev[i][j],prev[i+1][j],prev[i-1][j],prev[i][j+1],prev[i][j-1],f)

    solutions = next[::koef_x,::koef_y]

    print("Количество точек по x =", K)
    print("Количество точек по y =", L)
    print("N =", N)
    return solutions

### Таблицы результатов

In [198]:
u1 = analytic_points(6)
u1_df = pd.DataFrame(u1)


N = 10
k_x = 1
k_y = 1

x_grid = np.linspace(-1/2,1/2,6)
x_grid = np.round(x_grid, 2)
y_grid = np.linspace(0,1,6)

u2 = solve(k_x,k_y,N)
u2_df = pd.DataFrame(u2)

delta = np.zeros((6,6))
for j in range(6):
        for i in range(6):
            delta[i][j] = "{:.4e}".format(abs(u1[i][j] - u2[i][j]))


u = np.zeros((6,6))
u = u.astype(str)
for j in range(6):
    for i in range(6):
        u[i][j] = str("{:.4e}".format(u1[i][j])) + " | " + str("{:.4e}".format(u2[i][j]))
        
u = pd.DataFrame(u)
u_df = pd.DataFrame({**{"y/x": y_grid}, **{x_grid[i]:u[i] for i in range(6)}})
print("Первое число - аналитическое решение, второе - численное.")
display(u_df)

print("Максимальная норма ошибки =", "{:.4e}".format((max(map(max, delta)))))
delta_df = pd.DataFrame({**{"y/x": y_grid}, **{x_grid[i]:delta[i] for i in range(6)}})
display(delta_df)

Количество точек по x = 6
Количество точек по y = 6
N = 10
Первое число - аналитическое решение, второе - численное.


,y/x,-0.5,-0.3,-0.1,0.1,0.3,0.5
0,0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00
1,2.0000e-01,-0.0000e+00 | 0.0000e+00,-2.5600e-02 | -2.5297e-02,-3.8400e-02 | -3.7883e-02,-3.8400e-02 | -3.7883e-02,-2.5600e-02 | -2.5297e-02,-0.0000e+00 | 0.0000e+00
2,4.0000e-01,-0.0000e+00 | 0.0000e+00,-3.8400e-02 | -3.7883e-02,-5.7600e-02 | -5.6719e-02,-5.7600e-02 | -5.6719e-02,-3.8400e-02 | -3.7883e-02,-0.0000e+00 | 0.0000e+00
3,6.0000e-01,-0.0000e+00 | 0.0000e+00,-3.8400e-02 | -3.7883e-02,-5.7600e-02 | -5.6719e-02,-5.7600e-02 | -5.6719e-02,-3.8400e-02 | -3.7883e-02,-0.0000e+00 | 0.0000e+00
4,8.0000e-01,-0.0000e+00 | 0.0000e+00,-2.5600e-02 | -2.5297e-02,-3.8400e-02 | -3.7883e-02,-3.8400e-02 | -3.7883e-02,-2.5600e-02 | -2.5297e-02,-0.0000e+00 | 0.0000e+00
5,1.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00,0.0000e+00 | 0.0000e+00


Максимальная норма ошибки = 8.8085e-04


,y/x,-0.5,-0.3,-0.1,0.1,0.3,0.5
0,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
1,2.0000e-01,0.0000e+00,3.0331e-04,5.1742e-04,5.1742e-04,3.0331e-04,0.0000e+00
2,4.0000e-01,0.0000e+00,5.1742e-04,8.8085e-04,8.8085e-04,5.1742e-04,0.0000e+00
3,6.0000e-01,0.0000e+00,5.1742e-04,8.8085e-04,8.8085e-04,5.1742e-04,0.0000e+00
4,8.0000e-01,0.0000e+00,3.0331e-04,5.1742e-04,5.1742e-04,3.0331e-04,0.0000e+00
5,1.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
